## Function for Cross Validation

In [1]:
# Import StratifiedKFold
from sklearn.model_selection import StratifiedKFold
# Import confusion matrix, accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score

def cross_validate(model, X, y, cv=10):

    # Let's split data into 10 folds with stratisfied sampling
    kf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=0)

    # Let's create a dictionary to store the scores
    scores = {}
    
    # Let's iterate over the folds
    i = 1
    for train_index, test_index in kf.split(X, y):
        
        # Let's split the data into train and test
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Let's fit the model
        model.fit(X_train.values, y_train.values)
        
        # Let's predict the test data
        y_pred = model.predict(X_test.values)
        
        # Let's calculate the accuracy score
        acc = accuracy_score(y_test, y_pred)

        # Let's calculate the confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        
        # Let's append the score to the list
        scores[f'Fold-{i}'] = {'accuracy': acc, 'confusion_matrix': cm}
        i += 1

    # Let's return the scores
    return scores

## Decision Tree Class

Our Model can take the following hyper-parameters to avoid Overfitting:

*   max_depth: The maximum depth of the tree
*   min_entropy: The minimum entropy of any node else, we will consider it to be leaf
*   min_size: The minimum number of samples in a node else, we will consider it to be leaf

In [4]:
# Implemented Decision Tree Classifier without using sklearn module
import math
import numpy as np

class DTClassifier:

    def __init__(self, max_depth=None, min_entropy=None, min_size=None):
        
        # This will tell us about the condition on every node of the tree
        self.nodeValues = dict()
        
        # This is the root of the tree
        self.root = None

        # This dictonary maps the leaf node to the label
        self.out = dict()
        
        # This dictionary maps the leaf node to the probability of each label
        self.prob = dict()
        
        # This is the features of our dataset
        self.x_train = None
        
        # This is the labels of the y_train
        self.y_train = None
        
        # Maximum depth of the tree
        self.max_depth = max_depth
        
        # Limits the Minimum number of data points at some node
        self.min_size = min_size
        
        # Limits the Minimum entropy at some node
        self.min_entropy = min_entropy
        
        # Which paraeters to consider while checking the leaf node criteria
        self.leaf_methods = []
        
        # Total number of distinct classes
        self.total_distinct_classes = 0
        
        # Setting the leaf methods
        if max_depth is not None:
            self.leaf_methods.append('depth')
        if min_entropy is not None:
            self.leaf_methods.append('entropy')
        if min_size is not None:
            self.leaf_methods.append('size')
    
    def entropy(self, y):
        # Input: a list of labels
        # Output: the entropy of the list
        # Example:
        # Input: [0, 0, 1, 1, 1]
        # Output: 0.818

        if self.total_distinct_classes == 1:
          return 0

        # count the number of each label
        cnt = dict()
        for i in y:
            if i not in cnt.keys():
                cnt[i] = 1
            else:
                cnt[i] += 1

        # calculate the entropy
        entropy = 0

        for i in cnt.keys():
            # probability of each label
            p = cnt[i] / len(y)
            # entropy += -p * math.log(p, b)
            entropy += -p * math.log(p, self.total_distinct_classes)
        
        return entropy

    def fit(self, x_train, y_train, e = None, node = 1, depth = 0):
        # Input: x_train (binary vectors), y_train (labels)
        # Output: Builds the decision tree

        # If the node is the Root
        if e is None:
            
            # Setting the x_train and y_train
            self.x_train = x_train
            self.y_train = [x for x in y_train] # Note: y_train is a list of strings or any Immutable objects

            # Total number of distinct classes
            self.total_distinct_classes = len(np.unique(self.y_train))
            
            # Calculating the entropy of the whole dataset
            e = self.entropy(y_train)

        # If all labels are same
        if len(set(y_train)) == 1:
            
            # Probability of all labels
            probabilistic_label = dict()
            probabilistic_label[y_train[0]] = 1

            # Setting the probability of all labels which are not present at this leaf node
            for y in self.y_train:
                if y not in probabilistic_label.keys():
                    probabilistic_label[y] = 0
            
            # Setting the prob output for this leaf node
            self.prob[node] = probabilistic_label
            # Store the label with max probability in the out dictionary
            self.out[node] = max(probabilistic_label, key=probabilistic_label.get)

            return

        # If node matches the leaf node criteria
        if ('size' in self.leaf_methods and len(y_train) <= self.min_size) or ('entropy' in self.leaf_methods and e <= self.min_entropy) or ('depth' in self.leaf_methods and depth >= self.max_depth):
            
            # Probability of all labels
            probabilistic_label = dict()

            # Initializing the probabilistic label
            for y in self.y_train:
                probabilistic_label[y] = 0
            
            # Counting the number of times each label occurs
            for y in y_train:
                probabilistic_label[y] += 1
            
            # Normalizing the probabilities
            for y in probabilistic_label.keys():
                probabilistic_label[y] /= len(self.y_train)

            # Setting the probability of nodes which are not present at this leaf node
            for y in self.y_train:
                if y not in probabilistic_label.keys():
                    probabilistic_label[y] = 0
            
            # Setting the prob output for this leaf node
            self.prob[node] = probabilistic_label
            # Store the label with max probability in the out dictionary
            self.out[node] = max(probabilistic_label, key=probabilistic_label.get)
            
            return
        

        max_ig = -np.inf  # max information gain
        max_ig_index = -1  # index of the feature which has the max information gain

        ds1_x_best = []
        ds1_y_best = []
        ds2_x_best = []
        ds2_y_best = []
        
        # Consider splitting the dataset into two datasets based on feature number i belongs to [0, len(x_train[0])-1]
        for i in range(len(x_train[0])):
            
            # Splitting the dataset based on the feature number i
            ds1_x = []
            ds1_y = []
            ds2_x = []
            ds2_y = []

            # Splitting the dataset based on the feature number i
            for j in range(len(x_train)):
                if x_train[j][i] == 1:
                    ds1_y.append(y_train[j])
                    ds1_x.append(x_train[j])
                else:
                    ds2_y.append(y_train[j])
                    ds2_x.append(x_train[j])
            
            # If one of the child node is empty then do not split like this
            if len(ds1_y) == 0 or len(ds2_y) == 0:
                continue
            
            # Calculating the entropy of the two datasets
            e1 = self.entropy(ds1_y)
            e2 = self.entropy(ds2_y)

            # Calculating the information gain
            ig = e - (len(ds1_y) / len(y_train)) * e1 - (len(ds2_y) / len(y_train)) * e2

            # If the information gain is greater than the max information gain
            if ig > max_ig:
                max_ig = ig
                max_ig_index = i
                ds1_x_best = ds1_x
                ds1_y_best = ds1_y
                ds2_x_best = ds2_x
                ds2_y_best = ds2_y

        # Not splitting the dataset, this is also a leaf node
        if max_ig_index == -1:
            
            # Probability of all labels
            probabilistic_label = dict()

            # Initializing the probabilistic label
            for y in self.y_train:
                probabilistic_label[y] = 0
            
            # Counting the number of times each label occurs
            for y in y_train:
                probabilistic_label[y] += 1
            
            # Normalizing the probabilities
            for y in probabilistic_label.keys():
                probabilistic_label[y] /= len(self.y_train)

            # Setting the probability of nodes which are not present at this leaf node
            for y in self.y_train:
                if y not in probabilistic_label.keys():
                    probabilistic_label[y] = 0
            
            # Setting the prob output for this leaf node
            self.prob[node] = probabilistic_label
            # Store the label with max probability in the out dictionary
            self.out[node] = max(probabilistic_label, key=probabilistic_label.get)
            
            return
            
        # self.adjList[node] = [2*node, 2*node+1]
        self.nodeValues[node] = max_ig_index

        self.fit(ds1_x_best, ds1_y_best, e1, 2*node, depth+1)
        self.fit(ds2_x_best, ds2_y_best, e2, 2*node+1, depth+1)

    def predict(self, x_test):
        # Input: x_test (binary vectors), y_test (labels)
        # Output: a list of predicted labels
        y_pred = []
        for i in range(len(x_test)):
            out = self.predict_one(x_test[i])
            y_pred.append(out)
        return y_pred

    def predict_one(self, x_test):
        # Input: x_test (binary vector)
        # Output: a dictionary of probabilities of all labels
        node = 1 # Root node
        while node not in self.out.keys(): # While the node is not a leaf node
            if x_test[self.nodeValues[node]] == 1: # If the value of the feature is 1
                node = 2*node # Go to the Left child
            else:
                node = 2*node + 1 # Go to the Right child

        return self.out[node]
    
    def predict_proba(self, x_test):
        # Input: x_test (binary vectors), y_test (labels)
        # Output: a list of probabilities of all labels for each feature vector
        y_pred = []
        for i in range(len(x_test)):
            probs = self.predict_one_proba(x_test[i])
            y_pred.append(probs)
        
        return y_pred
    
    def predict_one_proba(self, x_test):
        # Input: x_test (binary vector)
        # Output: a dictionary of probabilities of all labels
        node = 1
        
        while node not in self.prob.keys():
            if x_test[self.nodeValues[node]] == 1:
                node = 2*node
            else:
                node = 2*node + 1
            
        return self.prob[node]


Using the Car Dataset provided in the HW

In [15]:
# decision tree 
import pandas as pd

age_of_driver = [25, 20, 25, 45, 20, 25]
car_type = ['Sports', 'Vintage', 'Sports', 'SUV', 'Sports', 'SUV']
risk = ['L', 'H', 'L', 'H', 'H', 'H']

In [16]:
# Create Dataframe
df = pd.DataFrame({'age_of_driver': age_of_driver, 'car_type': car_type, 'risk': risk})

In [17]:
df

,age_of_driver,car_type,risk
0,25,Sports,L
1,20,Vintage,H
2,25,Sports,L
3,45,SUV,H
4,20,Sports,H
5,25,SUV,H


Now, Considering 2 Splits:

1. Based on Car_Type
2. Age_Of_Driver

Since, we are using DT which works for only Categorical Variables, lets create a new variable

In [18]:
df['isAgeOfDriver22'] = df['age_of_driver'].apply(lambda x: 1 if x >= 22 else 0)

In [19]:
df

,age_of_driver,car_type,risk,isAgeOfDriver22
0,25,Sports,L,1
1,20,Vintage,H,0
2,25,Sports,L,1
3,45,SUV,H,1
4,20,Sports,H,0
5,25,SUV,H,1


In [20]:
df.drop('age_of_driver', axis=1, inplace=True)

In [21]:
df

,car_type,risk,isAgeOfDriver22
0,Sports,L,1
1,Vintage,H,0
2,Sports,L,1
3,SUV,H,1
4,Sports,H,0
5,SUV,H,1


In [22]:
# One Hot Encoding
df = pd.get_dummies(df, columns=['car_type'])


In [23]:
df

,risk,isAgeOfDriver22,car_type_SUV,car_type_Sports,car_type_Vintage
0,L,1,0,1,0
1,H,0,0,0,1
2,L,1,0,1,0
3,H,1,1,0,0
4,H,0,0,1,0
5,H,1,1,0,0


In [24]:
# Convert risk to 0 and 1
df['risk'] = df['risk'].apply(lambda x: 1 if x == 'H' else 0)

In [25]:
df

,risk,isAgeOfDriver22,car_type_SUV,car_type_Sports,car_type_Vintage
0,0,1,0,1,0
1,1,0,0,0,1
2,0,1,0,1,0
3,1,1,1,0,0
4,1,0,0,1,0
5,1,1,1,0,0


## Training the Decision Tree

In [27]:
model = DTClassifier()

In [28]:
# Train the model
model.fit(df.drop('risk', axis=1).values, df['risk'].values)

In [30]:
# Predicting for Rows in the same dataset
probs = model.predict_proba(df.drop('risk', axis=1).values)

In [31]:
for i in range(len(probs)):
    print('Probability of being High Risk for Row', i+1, ':', probs[i][1])

Probability of being High Risk for Row 1 : 0
Probability of being High Risk for Row 2 : 1
Probability of being High Risk for Row 3 : 0
Probability of being High Risk for Row 4 : 1
Probability of being High Risk for Row 5 : 1
Probability of being High Risk for Row 6 : 1


In [32]:
df

,risk,isAgeOfDriver22,car_type_SUV,car_type_Sports,car_type_Vintage
0,0,1,0,1,0
1,1,0,0,0,1
2,0,1,0,1,0
3,1,1,1,0,0
4,1,0,0,1,0
5,1,1,1,0,0


We can verify that our Model is working Correctly!